<a href="https://colab.research.google.com/github/Swastik200/30DaysOfDSA/blob/main/Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyMetaheuristic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 1.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from pyMetaheuristic.algorithm import genetic_algorithm
from pyMetaheuristic.utils import graphs

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp "/content/drive/MyDrive/CMAPSS.zip" "/content"
!unzip CMAPSS.zip

Archive:  CMAPSS.zip
  inflating: CMAPSS/readme.txt       
  inflating: CMAPSS/RUL_FD001.txt    
  inflating: CMAPSS/RUL_FD002.txt    
  inflating: CMAPSS/RUL_FD003.txt    
  inflating: CMAPSS/RUL_FD004.txt    
  inflating: CMAPSS/test_FD001.txt   
  inflating: CMAPSS/test_FD002.txt   
  inflating: CMAPSS/test_FD003.txt   
  inflating: CMAPSS/test_FD004.txt   
  inflating: CMAPSS/train_FD001.txt  
  inflating: CMAPSS/train_FD002.txt  
  inflating: CMAPSS/train_FD003.txt  
  inflating: CMAPSS/train_FD004.txt  
  inflating: CMAPSS/x.txt            


In [5]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['sensor_{}'.format(i) for i in range(1, 22)]
col_names = index_names + setting_names + sensor_names

In [6]:
path = '/content/CMAPSS/'
df_train = pd.read_csv(path + 'train_FD001.txt', sep='\s+', header=None, index_col=False, names=col_names)
df_test = pd.read_csv(path + 'test_FD001.txt', sep='\s+', header=None, index_col=False, names=col_names)
y_test = pd.read_csv(path + 'RUL_FD001.txt', sep='\s+', header=None, index_col=False, names=['RUL'])

In [8]:
# Calculate 'RUL' column based on 'time_cycles' and 'unit_number' columns
max_cycles = df_train.groupby('unit_number')['time_cycles'].max()
df_train['RUL'] = df_train.apply(lambda row: max_cycles[row['unit_number']] - row['time_cycles'], axis=1)
# Calculate 'RUL' column based on 'time_cycles' and 'unit_number' columns
max_cycles = df_test.groupby('unit_number')['time_cycles'].max()
df_test['RUL'] = df_test.apply(lambda row: max_cycles[row['unit_number']] - row['time_cycles'], axis=1)


In [9]:
X_train = df_train.drop(columns=['RUL'])
y_train = df_train['RUL']
X_test = df_test.drop(columns=['RUL'])
y_test = y_test['RUL']


In [10]:
parameters = {
    'population_size': 250,
    'min_values': [-5] * len(X_train.columns),  # Adjust according to your feature space
    'max_values': [5] * len(X_train.columns),  # Adjust according to your feature space
    'generations': 10,
    'mutation_rate': 0.1,
    'elite': 1,
    'eta': 1,
    'mu': 1,
    'verbose': True,
    'start_init': None,
    'target_value': None
}

In [11]:
def target_function(variables_values):
    # Train the regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Predict using the trained model
    predictions = model.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    return rmse

In [14]:
# Assuming you have already loaded your dataset into variables df_train and df_test

# Define X and y for the training dataset
X_train = df_train.drop(columns=['RUL'])  # Features
y_train = df_train['RUL']  # Target variable

# Define X and y for the testing dataset
X_test = df_test.drop(columns=['RUL'])  # Features
y_test = df_test['RUL']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [15]:
ga_result = genetic_algorithm(target_function=target_function, **parameters)

Generation =  0  f(x) =  39.22801459451258
Generation =  1  f(x) =  39.22801459451258
Generation =  2  f(x) =  39.22801459451258
Generation =  3  f(x) =  39.22801459451258
Generation =  4  f(x) =  39.22801459451258
Generation =  5  f(x) =  39.22801459451258
Generation =  6  f(x) =  39.22801459451258
Generation =  7  f(x) =  39.22801459451258
Generation =  8  f(x) =  39.22801459451258
Generation =  9  f(x) =  39.22801459451258
Generation =  10  f(x) =  39.22801459451258


In [16]:
optimized_model = LinearRegression().fit(X_test, y_test)

In [17]:
rmse_test = np.sqrt(mean_squared_error(y_test, optimized_model.predict(X_test)))
r2_test = r2_score(y_test, optimized_model.predict(X_test))
mae_test = mean_absolute_error(y_test, optimized_model.predict(X_test))

In [18]:
print('RMSE (Test):', rmse_test)
print('R2 Score (Test):', r2_test)
print('MAE (Test):', mae_test)

RMSE (Test): 38.96712711655703
R2 Score (Test): 0.6676504517530486
MAE (Test): 30.025205313555507
